In [ ]:
from google.colab import drive

# Accessing My Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


## Unzip dataset on runtime session

In [ ]:
!unzip '/content/drive/My Drive/CSE331_ML_Project/dataset.zip'

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: valid/truck/n03632852_1173.png  
  inflating: valid/truck/n03632852_11766.png  
  inflating: valid/truck/n03632852_11887.png  
  inflating: valid/truck/n03632852_1191.png  
  inflating: valid/truck/n03632852_1192.png  
  inflating: valid/truck/n03632852_1195.png  
  inflating: valid/truck/n03632852_1207.png  
  inflating: valid/truck/n03632852_12128.png  
  inflating: valid/truck/n03632852_1214.png  
  inflating: valid/truck/n03632852_1237.png  
  inflating: valid/truck/n03632852_1248.png  
  inflating: valid/truck/n03632852_1261.png  
  inflating: valid/truck/n03632852_1264.png  
  inflating: valid/truck/n03632852_1292.png  
  inflating: valid/truck/n03632852_1313.png  
  inflating: valid/truck/n03632852_1319.png  
  inflating: valid/truck/n03632852_13311.png  
  inflating: valid/truck/n03632852_1355.png  
  inflating: valid/truck/n03632852_13648.png  
  inflating: valid/truck/n03632852_13730.png  
  inflating: valid/truck/n03632852_138

## Define Model

In [ ]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import glob
import PIL
from PIL import Image
from torch.utils import data as D
from torch.utils.data.sampler import SubsetRandomSampler
import random
import torchsummary

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

class Bottleneck(nn.Module):
  def __init__(self, in_channels, growth_rate, drop_rate=0.2):
    super().__init__()
    inner_channel = 4 * growth_rate
    self.drop_rate=drop_rate

    self.bottle_neck = nn.Sequential(
        nn.BatchNorm2d(in_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels, inner_channel, kernel_size=1, bias=False),
        nn.BatchNorm2d(inner_channel),
        nn.ReLU(inplace=True),
        nn.Conv2d(inner_channel, growth_rate, kernel_size=3, padding=1, bias=False)
        )

  def forward(self, x):
    
    bottleneck_output=self.bottle_neck(x)

    if self.drop_rate > 0:
      bottleneck_output = F.dropout(bottleneck_output, p=self.drop_rate, training=self.training)
      
    return torch.cat([x, bottleneck_output], 1)

class Transition(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()
    self.down_sample = nn.Sequential(
          nn.BatchNorm2d(in_channels),
          nn.Conv2d(in_channels, out_channels, 1, bias=False),
          nn.AvgPool2d(2, stride=2)
      )

  def forward(self, x):
    return self.down_sample(x)

class DenseNet(nn.Module):
  def __init__(self, block, nblocks, growth_rate=12, reduction=0.5, num_class=10, drop_rate=0.2):
      super().__init__()
      self.growth_rate = growth_rate
      inner_channels = 2 * growth_rate
      self.conv1 = nn.Conv2d(3, inner_channels, kernel_size=3, padding=1, bias=False) 

      self.features = nn.Sequential()

      for index in range(len(nblocks) - 1):
          self.features.add_module("dense_block_layer_{}".format(index), self._make_dense_layers(block, inner_channels, nblocks[index]))
          inner_channels += growth_rate * nblocks[index]
          out_channels = int(reduction * inner_channels)
          self.features.add_module("transition_layer_{}".format(index), Transition(inner_channels, out_channels))
          inner_channels = out_channels

      self.features.add_module("dense_block{}".format(len(nblocks) - 1), self._make_dense_layers(block, inner_channels, nblocks[len(nblocks)-1]))
      inner_channels += growth_rate * nblocks[len(nblocks) - 1]
      self.features.add_module('bn', nn.BatchNorm2d(inner_channels))
      self.features.add_module('activation', nn.ReLU(inplace=True))

      self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

      self.linear = nn.Linear(inner_channels, num_class)

  def forward(self, x):
      output = self.conv1(x)
      output = self.features(output)
      output = self.avgpool(output)
      output = output.view(output.size()[0], -1)
      output = self.linear(output)
      return output

  def _make_dense_layers(self, block, in_channels, nblocks):
      dense_block = nn.Sequential()
      for index in range(nblocks):
          dense_block.add_module('bottle_neck_layer_{}'.format(index), block(in_channels, self.growth_rate))
          in_channels += self.growth_rate
      return dense_block

def densenet121(activation = 'relu'):
  return DenseNet(Bottleneck, [6,9,12,15,18,21], growth_rate=12 ,drop_rate=0.2)

def densenet169(activation = 'relu'):
  return DenseNet(Bottleneck, [6,12,32,32], growth_rate=24)

def densenet201(activation = 'relu'):
  return DenseNet(Bottleneck, [6,12,48,32], growth_rate=32)

def densenet161(activation = 'relu'):
  return DenseNet(Bottleneck, [6,12,36,24], growth_rate=24)


model = densenet121()
model.to(device)

def Model():
  r"""Return your custom model
  """
  return model

cuda:0


## utils

In [ ]:
class AverageMeter(object):
    r"""Computes and stores the average and current value
    """
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    r"""Computes the accuracy over the $k$ top predictions for the specified values of k
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # _, pred = output.topk(maxk, 1, True, True)
        # pred = pred.t()
        # correct = pred.eq(target.view(1, -1).expand_as(pred))

        # faster topk (ref: https://github.com/pytorch/pytorch/issues/22812)
        _, idx = output.sort(descending=True)
        pred = idx[:,:maxk]
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []

        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

## Hyperparameters

In [ ]:
SAVEPATH = '/content/drive/My Drive/CSE331_ML_Project/'
WEIGHTDECAY = 1e-4
MOMENTUM = 0.9
BATCHSIZE = 256
LR = 0.001
EPOCHS = 200
PRINTFREQ = 100

## Train Model

In [ ]:
import time

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd

epochs = [x for x in range(EPOCHS)]
train_stats = pd.DataFrame(index = epochs , columns = ['Train loss', 'Train accuracy','Valid loss', 'Valid accuracy'])

n_epochs_stop = 1000
epochs_no_improve = 0
early_stop = False
min_val_loss = 999
val_loss=0

def main():

    model = Model()
    #model.load_state_dict(torch.load(SAVEPATH+"densenet201.pth"))
    ##### optimizer / learning rate scheduler / criterion #####
    optimizer = torch.optim.Adam(model.parameters(), lr=LR,
                                weight_decay=WEIGHTDECAY)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [100, 150],
                                                     gamma=0.1)
    #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    criterion = torch.nn.CrossEntropyLoss()
    ###########################################################

    model = model.cuda()
    criterion = criterion.cuda()

    # Check number of parameters your model
    pytorch_total_params = sum(p.numel() for p in model.parameters())
    print(f"Number of parameters: {pytorch_total_params}")
    if int(pytorch_total_params) > 2000000:
        print('Your model has the number of parameters more than 2 millions..')
        return

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])

    valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])

    #
    train_dataset = torchvision.datasets.ImageFolder('./train', transform = train_transform)
    #valid dataset에서 transfrom 안하고 가져옴
    valid_dataset = torchvision.datasets.ImageFolder('./valid')

    #valid dataset에서 18000개는 validation을 위해서 남겨둠
    valid1, valid2 = torch.utils.data.random_split(valid_dataset,[80000,10000])

  
    #먼저 vallid2(validation에 사용)를 valid_transform을 적용해서 텐서로 변환하고 정규화
    tmp1=[]
    for i in range(10000):
      valid2_img = (valid_transform((valid2[i][0])),valid2[i][1])
      tmp1.append(valid2_img)
    
    valid2 = tuple(tmp1)

    #valid1에 train_transform을 적용해준다
    tmp2=[]
    for i in range(80000):
      valid1_img = (train_transform((valid1[i][0])),valid1[i][1])
      tmp2.append(valid1_img)

    valid1 = tuple(tmp2)

    #train_dataset과 valid1을 합쳐준다
    train_dataset += valid1


    train_loader = DataLoader(train_dataset,
                              batch_size=BATCHSIZE, shuffle=True,
                              num_workers=4, pin_memory=True)
    
    val_loader = DataLoader(valid2, batch_size=BATCHSIZE, num_workers=4, shuffle=False)

    last_top1_acc = 0
    for epoch in range(EPOCHS):
        
        print("\n----- epoch: {}, lr: {} -----".format(
            epoch, optimizer.param_groups[0]["lr"]))

        # train for one epoch
        start_time = time.time()
        last_top1_acc = train(train_loader, epoch, model, optimizer, criterion)
        elapsed_time = time.time() - start_time
        print('==> {:.2f} seconds to train this epoch\n'.format(
            elapsed_time))

        # learning rate scheduling
        scheduler.step()

        # Save model each epoch
        torch.save(model.state_dict(), SAVEPATH+'dense_deep_1209.pth')
        last_top1_val_acc = valid(val_loader, epoch, model, criterion)

        #scheduler.step(val_loss)
        if early_stop:
          print("Stopped")
          break
        train_stats.loc[epoch]['Train accuracy'] = last_top1_acc
        train_stats.loc[epoch]['Valid accuracy'] = last_top1_val_acc

        train_stats.to_csv(SAVEPATH+'train_log_dense_deep_1209.csv')

    print(f"Last Top-1 Accuracy: {last_top1_acc}")
    print(f"Number of parameters: {pytorch_total_params}")



def train(train_loader, epoch, model, optimizer, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(len(train_loader), batch_time, data_time, losses,
                             top1, top5, prefix="Epoch: [{}]".format(epoch))
    # switch to train mode
    model.train()

    end = time.time()


    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        input = input.cuda()
        target = target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss, accuracy 
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(acc1[0].item(), input.size(0))
        top5.update(acc5[0].item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINTFREQ == 0:
            progress.print(i)

    print('=> Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))
    train_stats.loc[epoch]['Train loss'] = losses.avg
    return top1.avg

def valid(val_loader, epoch, model, criterion):
  global n_epochs_stop
  global epochs_no_improve
  global early_stop
  global min_val_loss 
  global val_loss
  val_loss =0
  

  print("Try Validation...")
  model.eval()
  batch_time = AverageMeter('Time', ':6.3f')
  data_time = AverageMeter('Data', ':6.3f')
  losses = AverageMeter('Val Loss', ':.4e')
  top1 = AverageMeter('Val Acc@1', ':6.2f')
  top5 = AverageMeter('Val Acc@5', ':6.2f')
  progress = ProgressMeter(len(val_loader), batch_time, data_time, losses,
                            top1, top5, prefix="Epoch: [{}]".format(epoch))
  end = time.time()
  for i, (input, target) in enumerate(val_loader):
      # measure data loading time
      data_time.update(time.time() - end)

      input = input.cuda()
      target = target.cuda()

      # compute output
      output = model(input)
      loss = criterion(output, target)
      val_loss += loss.item()

      # measure accuracy and record loss, accuracy 
      acc1, acc5 = accuracy(output, target, topk=(1, 5))
      losses.update(loss.item(), input.size(0))
      top1.update(acc1[0].item(), input.size(0))
      top5.update(acc5[0].item(), input.size(0))
      # compute gradient and do SGD step
      # measure elapsed time
      batch_time.update(time.time() - end)
      end = time.time()

      if i % 100 * PRINTFREQ == 0:
          progress.print(i)

  val_loss = val_loss / len(val_loader)
  if val_loss < min_val_loss:
    min_val_loss = val_loss
    # Save model this epoch
    torch.save(model.state_dict(), SAVEPATH+'dense_deep_1209_best.pth')
    epochs_no_improve = 0
  else:
    epochs_no_improve += 1
    if epochs_no_improve == n_epochs_stop:
      print('Early stopping!')
      early_stop = True



  print('=> Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
        .format(top1=top1, top5=top5))
  print('Avg val_oss : {}'.format(val_loss))
  print("epoch_no_improve: {}" .format(epochs_no_improve))
  print("min val loss: {}" .format(min_val_loss))

  train_stats.loc[epoch]['Valid loss'] = losses.avg

  return top1.avg

if __name__ == "__main__":
    main()

Number of parameters: 1413717

----- epoch: 0, lr: 0.001 -----
Epoch: [0][  0/665]	Time  1.130 ( 1.130)	Data  0.536 ( 0.536)	Loss 2.3952e+00 (2.3952e+00)	Acc@1   9.77 (  9.77)	Acc@5  49.61 ( 49.61)
Epoch: [0][100/665]	Time  0.229 ( 0.243)	Data  0.000 ( 0.005)	Loss 1.6815e+00 (1.8143e+00)	Acc@1  39.84 ( 31.65)	Acc@5  86.72 ( 84.24)
Epoch: [0][200/665]	Time  0.216 ( 0.235)	Data  0.000 ( 0.003)	Loss 1.5330e+00 (1.6840e+00)	Acc@1  42.58 ( 36.84)	Acc@5  92.19 ( 87.82)
Epoch: [0][300/665]	Time  0.218 ( 0.234)	Data  0.000 ( 0.002)	Loss 1.3048e+00 (1.6016e+00)	Acc@1  46.48 ( 40.10)	Acc@5  95.70 ( 89.43)
Epoch: [0][400/665]	Time  0.207 ( 0.231)	Data  0.000 ( 0.002)	Loss 1.2179e+00 (1.5363e+00)	Acc@1  50.00 ( 42.68)	Acc@5  95.70 ( 90.57)
Epoch: [0][500/665]	Time  0.222 ( 0.230)	Data  0.000 ( 0.001)	Loss 1.1582e+00 (1.4858e+00)	Acc@1  58.20 ( 44.76)	Acc@5  97.66 ( 91.35)
Epoch: [0][600/665]	Time  0.208 ( 0.229)	Data  0.000 ( 0.001)	Loss 1.3577e+00 (1.4433e+00)	Acc@1  52.34 ( 46.47)	Acc@5  92.19 (

## Make an evalutation csv file

This code makes an evaluation csv file for kaggle submission.


In [ ]:
import torch
import pandas as pd
import argparse
import time
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

class TestImageFolder(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        # return image path
        return super(TestImageFolder, self).__getitem__(index), self.imgs[index][0].split('/')[-1]


def eval():
    ########## You can change this part only in this cell ##########
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])
    ################################################################

    test_dataset = TestImageFolder('./test', transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCHSIZE, num_workers=4, shuffle=False)

    model = Model()
    model = model.cuda()
    model.load_state_dict(torch.load(SAVEPATH+'dense_deep_1209.pth'))
    model.eval()

    print('Make an evaluation csv file for kaggle submission...')
    Category = []
    Id = []
    for data in test_loader:
        (input, _), name = data
        
        input = input.cuda()
        output = model(input)
        output = torch.argmax(output, dim=1)
        Id = Id + list(name)
        Category = Category + output.tolist()

    #Id = list(range(0, 90000))
    samples = {
       'Id': Id,
       'Target': Category 
    }
    df = pd.DataFrame(samples, columns=['Id', 'Target'])

    df.to_csv(SAVEPATH+'dense_deep_1209_submission.csv', index=False)
    print('Done!!')


if __name__ == "__main__":
    eval()

Make an evaluation csv file for kaggle submission...
Done!!
